# **Manejo de Valores Perdidos en Python: Diagnóstico e Imputación por Regresión**
---
*Autor: Francisco Díaz*

Este documento se ha elaborado con fines didácticos. En cualquier conjunto de datos, los valores perdidos se refieren a aquellos datos que se esperaba estuvieran presentes, pero que, por diversas razones, no están disponibles o no se registraron durante la recopilación de datos. Estas ausencias pueden ser el resultado de errores en la recopilación de datos o de respuestas no proporcionadas en una encuesta.

El diagnóstico y el tratamiento de los valores perdidos son cruciales por varias razones:

+ **Integridad de los datos:** Los valores perdidos pueden afectar la calidad y la integridad de los datos.
+ **Análisis estadístico:** La presencia de valores perdidos puede dificultar o incluso imposibilitar la realización de ciertos análisis.
+ **Sesgo en los resultados:** Si los valores perdidos no son aleatorios (es decir, si existe una razón sistemática por la que algunos datos están ausentes), esto puede introducir un sesgo en los resultados del análisis.

Este cuaderno tiene como objetivo explorar el diagnóstico de valores perdidos mediante correlaciones dicotómicas. Posteriormente, se realizará una imputación de los valores ausentes utilizando la regresión lineal. Es importante mencionar que los datos utilizados en este cuaderno provienen de una encuesta ficticia sobre el hábito de fumar tabaco.

A continuación, se presentan los metadatos del conjunto de datos:

|Variable|Descripción|
|--------|-----------|
|V1| Fumar es dañino para la salud|
|V2| No debe permitirse fumar en lugares publicos |
|V3| A las instituciones publicas solo les interesa recaudar impuestos |
|V4| Deben aumentarse los impuestos al tabaco |
|V5| Debe informarse mas sobre los daños del tabaco |
|C1| Edad en años |
|C2| Sexo (Hombre, Mujer)|
|C3| Habito (Fumador, No Fumador) |

La variable `v4a` representa respuestas aleatorias a la pregunta *“¿Deben aumentarse los impuestos al tabaco?”*, mientras que la variable `v4b` muestra respuestas que siguen un patrón sistemático. A partir de estas variables, se crearon las variables codificadas `v4a_d` y `v4b_d`, que indican si los valores son *“Perdidos”* o *“No perdidos”*.

Siguiendo la misma lógica, se creó la variable `v2_d`, que es una variable **dummy** de `v2` *“No debe permitirse fumar en lugares públicos”*. Esta variable se utilizará para diagnosticar los valores perdidos de `v4a_d` y `v4b_d`, con el objetivo de determinar si existen patrones en los valores perdidos o si estos son aleatorios. Es importante destacar que buscamos aleatoriedad en los valores perdidos.

Librerias a utilizar:

In [1]:
!pip install pyreadstat # Para leer archivos SPSS
import pandas as pd # Para manipular datos
import statsmodels.api as sm # Para crear regresiones
from scipy.stats import pearsonr # Se usa para crear matrices de correlaciones

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 19.9 MB/s eta 0:00:00


Importamos el conjunto de datos

In [2]:
df = pd.read_spss("/content/drive/MyDrive/Datos_2_1_Caso.sav") # Importamos
df.head() # Muestra las primeras 5 filas

,caso,v1,v2,v3,v4a,v4b,v5,c1,c2,c3,v4a_d,v4b_d,v2_d
0,1.0,5.0,5.0,4.0,NaN,5.0,5.0,21.0,Mujer,No fumador,Perdidos,No perdidos,No perdidos
1,2.0,5.0,5.0,4.0,4.0,4.0,5.0,21.0,Mujer,No fumador,No perdidos,No perdidos,No perdidos
2,3.0,5.0,5.0,4.0,2.0,2.0,5.0,21.0,Hombre,No fumador,No perdidos,No perdidos,No perdidos
3,4.0,5.0,4.0,3.0,NaN,3.0,4.0,20.0,Mujer,No fumador,Perdidos,No perdidos,No perdidos
4,5.0,5.0,5.0,2.0,5.0,5.0,5.0,24.0,Mujer,No fumador,No perdidos,No perdidos,No perdidos


## Tecnica de correlaciones dicotomizadas
En situaciones prácticas, resulta desafiante discernir si los valores ausentes obedecen a un patrón sistemático o si se deben a un comportamiento aleatorio. En esta sección, implementaremos una técnica de correlaciones dicotomizadas, con el objetivo de identificar la posible existencia de un patrón en los valores faltantes. Esta técnica, consiste en el uso de una matriz de correlaciones y de p-valores.

In [3]:
# Codificamos los valores
df['v4b_d'] = df['v4b_d'].map({"No perdidos": 1, "Perdidos": 0})
df['v2_d'] = df['v2_d'].map({"No perdidos": 1, "Perdidos": 0})
df['v4a_d'] = df['v4a_d'].map({"No perdidos": 1, "Perdidos": 0})

In [4]:
# Convertimos los valores en tipo "int"
df["v4a_d"]=df["v4a_d"].astype(int)
df["v4b_d"]=df["v4b_d"].astype(int)
df["v2_d"]=df["v2_d"].astype(int)

In [5]:
df.head() # Muestra las primeras 5 filas

,caso,v1,v2,v3,v4a,v4b,v5,c1,c2,c3,v4a_d,v4b_d,v2_d
0,1.0,5.0,5.0,4.0,NaN,5.0,5.0,21.0,Mujer,No fumador,0,1,1
1,2.0,5.0,5.0,4.0,4.0,4.0,5.0,21.0,Mujer,No fumador,1,1,1
2,3.0,5.0,5.0,4.0,2.0,2.0,5.0,21.0,Hombre,No fumador,1,1,1
3,4.0,5.0,4.0,3.0,NaN,3.0,4.0,20.0,Mujer,No fumador,0,1,1
4,5.0,5.0,5.0,2.0,5.0,5.0,5.0,24.0,Mujer,No fumador,1,1,1


In [6]:
df.describe() # Obtenemos las estadisticas descriptivas del dataframe

,caso,v1,v2,v3,v4a,v4b,v5,c1,v4a_d,v4b_d,v2_d
count,30.000000,30.000000,26.000000,30.00000,24.000000,23.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,15.500000,4.666667,4.153846,3.80000,2.916667,3.434783,4.433333,22.066667,0.800000,0.766667,0.866667
std,8.803408,0.922266,0.833897,1.15669,1.529895,1.375965,0.817200,1.760355,0.406838,0.430183,0.345746
min,1.000000,1.000000,2.000000,1.00000,1.000000,1.000000,3.000000,20.000000,0.000000,0.000000,0.000000
25%,8.250000,5.000000,4.000000,3.00000,1.000000,2.500000,4.000000,21.000000,1.000000,1.000000,1.000000
50%,15.500000,5.000000,4.000000,4.00000,3.000000,4.000000,5.000000,22.000000,1.000000,1.000000,1.000000
75%,22.750000,5.000000,5.000000,5.00000,4.000000,4.500000,5.000000,23.000000,1.000000,1.000000,1.000000
max,30.000000,5.000000,5.000000,5.00000,5.000000,5.000000,5.000000,27.000000,1.000000,1.000000,1.000000


Se observa una correlación fuerte y positiva entre `v4b_d`, que representa los valores no aleatorios de valores perdidos de `V4`, y `v2_d`. Esto sugiere que es muy probable que las mismas personas que no respondieron a la pregunta `V2` tampoco lo hicieran en `v4b_d`

In [7]:
df[['v2_d','v4a_d', 'v4b_d']].corr(method='pearson')

,v2_d,v4a_d,v4b_d
v2_d,1.000000,-0.196116,0.710981
v4a_d,-0.196116,1.000000,0.118217
v4b_d,0.710981,0.118217,1.000000


En la matriz de p-valores, se determina que la correlación de 0.710981 entre `v4b_d` y `v2_d` es estadísticamente significativa. Esto indica un comportamiento sistemático en `v4b_d`.

Por otra parte, dado que `v4a_d` no muestra una correlación significativa con los valores perdidos de `v2` (`v2_d`) este resultado permite indicar que la generación de valores perdidos ha sido aleatoria para `v4`.

In [8]:
def pearsonr_pval(x,y):
          return pearsonr(x,y)[1]
df[['v2_d','v4a_d', 'v4b_d']].corr(method=pearsonr_pval)

,v2_d,v4a_d,v4b_d
v2_d,1.000000,0.298964,0.000011
v4a_d,0.298964,1.000000,0.533828
v4b_d,0.000011,0.533828,1.000000


## Imputación por regresión
El tratamiento de valores perdidos se puede realizar mediante la eliminación de estos valores.

En cuanto a la imputación de valores perdidos se puede usar la media de la variable de los valores faltantes o con el uso de la mediana de la variable. Cada método tiene sus ventajas y desventajas.

En esta sección, utilizaremos la regresión lineal para imputar los valores faltantes.

In [9]:
# Eliminamos las filas donde la columna 'v4a' tiene valores perdidos
df_clean = df.dropna(subset=['v4a'])

# Corremos la regresión
y = df_clean['v4a']
X = df_clean[['v1', 'v3', 'v5', 'c1']]
X = sm.add_constant(X)
modelo = sm.OLS(y, X).fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                    v4a   R-squared:                       0.467
Model:                            OLS   Adj. R-squared:                  0.355
Method:                 Least Squares   F-statistic:                     4.164
Date:                Wed, 10 Apr 2024   Prob (F-statistic):             0.0137
Time:                        04:30:32   Log-Likelihood:                -36.195
No. Observations:                  24   AIC:                             82.39
Df Residuals:                      19   BIC:                             88.28
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0162      3.670     -0.004      0.9

In [11]:
df[df['v4a'].isna()] # identificamos los perdidos

,caso,v1,v2,v3,v4a,v4b,v5,c1,c2,c3,v4a_d,v4b_d,v2_d
0,1.0,5.0,5.0,4.0,NaN,5.0,5.0,21.0,Mujer,No fumador,0,1,1
3,4.0,5.0,4.0,3.0,NaN,3.0,4.0,20.0,Mujer,No fumador,0,1,1
9,10.0,5.0,2.0,3.0,NaN,NaN,3.0,21.0,Mujer,Fumador,0,0,1
17,18.0,5.0,4.0,4.0,NaN,NaN,3.0,20.0,Hombre,Fumador,0,0,1
27,28.0,4.0,4.0,5.0,NaN,2.0,5.0,22.0,Mujer,No fumador,0,1,1
28,29.0,5.0,4.0,4.0,NaN,5.0,5.0,22.0,Hombre,Fumador,0,1,1


In [14]:
# Aplicamos coeficientes de la regresión
caso1 = -0.0162 + 0.1374*5-0.3720*4+0.9265*5-0.0206*21
print(round(caso1,0))
caso4 = -0.0162 + 0.1374*5-0.3720*3+0.9265*4-0.0206*20
print(round(caso4,0))

3.0
3.0


In [ ]:

# Reemplazar valores perdidos por resultados de la regresión
valores_especificos = [3, 3, 3, 5]
df['v4a'].fillna(pd.Series(valores_especificos), inplace=True)

In [ ]:
# hacer regresión para v2

## Conjunto de datos limpio
Al aplicar las tecnicas de diagnostico y tratamiento de valores perdidos, se obtiene el siguiente conjunto de datos limpio.

In [ ]:
df.drop(columns=['v4b', 'v4a_d', 'v4b_d', 'v2_d'], inplace=True)

In [ ]:
df.isna().sum()

caso    0
v1      0
v2      4
v3      0
v4a     4
v5      0
c1      0
c2      0
c3      0
dtype: int64

In [ ]:
df.isna().sum()

In [ ]:
df.info() #final

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   caso    30 non-null     float64 
 1   v1      30 non-null     float64 
 2   v2      26 non-null     float64 
 3   v3      30 non-null     float64 
 4   v4a     26 non-null     float64 
 5   v5      30 non-null     float64 
 6   c1      30 non-null     float64 
 7   c2      30 non-null     category
 8   c3      30 non-null     category
dtypes: category(2), float64(7)
memory usage: 2.1 KB


## Bibliografia